In [94]:
import pandas as pd
import json
import ast

In [95]:
df = pd.read_csv("raw/megasena.csv")

# megasena

In [96]:
# Configurações de diferentes loterias
expected_count = 6

## Limpeza e ajustes nos dados

In [97]:
df = df.reset_index()

In [98]:
## Criar condição aqui
df.drop(columns=['timeCoracao', 'mesSorte', 'trevos'], inplace=True)

In [99]:
df.head()

,index,Unnamed: 0,loteria,concurso,data,local,dezenasOrdemSorteio,dezenas,premiacoes,estadosPremiados,observacao,acumulou,proximoConcurso,dataProximoConcurso,localGanhadores,valorArrecadado,valorAcumuladoConcurso_0_5,valorAcumuladoConcursoEspecial,valorAcumuladoProximoConcurso,valorEstimadoProximoConcurso
0,0,0,megasena,1551.0,27/11/2013,"Auditório em OSASCO, SP","['02', '23', '38', '15', '21', '19']","['02', '15', '19', '21', '23', '38']","[{'descricao': '6 acertos', 'faixa': 1, 'ganha...",[],Sorteio realizado em Estúdio de TV.,0.0,1552.0,30/11/2013,"[{'ganhadores': 1, 'municipio': 'BOA VISTA', '...",26151116.0,4782195.52,5.571652e+07,0.00,3000000.0
1,1,2,megasena,664.0,14/05/2005,"Caminhão da Sorte em PASSOS, MG","['12', '01', '55', '59', '38', '36']","['01', '12', '36', '38', '55', '59']","[{'descricao': '6 acertos', 'faixa': 1, 'ganha...",[],ACUMULOU!!! Estimativa de prêmio (SENA) próxim...,1.0,665.0,18/05/2005,[],0.0,23857740.39,0.000000e+00,23857740.39,0.0
2,2,4,megasena,366.0,01/06/2002,"Caminhão da Sorte em Orlândia, SP","['06', '04', '51', '07', '21', '15']","['04', '06', '07', '15', '21', '51']","[{'descricao': '6 acertos', 'faixa': 1, 'ganha...",[],Estimativa de prêmio (SENA) próximo concurso: ...,1.0,367.0,05/06/2002,[],0.0,5048832.63,0.000000e+00,625880.91,0.0
3,3,5,megasena,537.0,11/02/2004,"Caminhão da Sorte em IBAITI, PR","['37', '03', '19', '34', '39', '07']","['03', '07', '19', '34', '37', '39']","[{'descricao': '6 acertos', 'faixa': 1, 'ganha...",[],Estimativa de prêmio (SENA) próximo concurso: ...,0.0,538.0,14/02/2004,"[{'ganhadores': 1, 'municipio': '', 'nomeFatan...",0.0,2354230.53,0.000000e+00,0.00,0.0
4,4,7,megasena,2786.0,16/10/2024,"ESPAÇO DA SORTE em SÃO PAULO, SP","['06', '17', '11', '51', '20', '40']","['06', '11', '17', '20', '40', '51']","[{'descricao': '6 acertos', 'faixa': 1, 'ganha...",[],NaN,1.0,2787.0,19/10/2024,[],66546110.0,8337773.84,1.076991e+08,33761273.32,42000000.0


In [100]:
df.columns

Index(['index', 'Unnamed: 0', 'loteria', 'concurso', 'data', 'local',
       'dezenasOrdemSorteio', 'dezenas', 'premiacoes', 'estadosPremiados',
       'observacao', 'acumulou', 'proximoConcurso', 'dataProximoConcurso',
       'localGanhadores', 'valorArrecadado', 'valorAcumuladoConcurso_0_5',
       'valorAcumuladoConcursoEspecial', 'valorAcumuladoProximoConcurso',
       'valorEstimadoProximoConcurso'],
      dtype='object')

### removendo concursos duplicados

In [101]:
if df.duplicated(subset=["concurso"]).sum():
    df = df.drop_duplicates(subset=["concurso"], keep="last")

### corrigindo tipagem de colunas concurso e data (dayfirst por conta do formato brasileiro e mixed pois existem 27/02/2025 e 27-02-2025)

In [102]:
df["concurso"] = pd.to_numeric(df["concurso"], errors="coerce").astype("Int64")

In [103]:
df["data"] = pd.to_datetime(df["data"], dayfirst=True, format='mixed')
df["proximoConcurso"] = pd.to_datetime(df["proximoConcurso"], dayfirst=True, format='mixed')

### corrigindo tipagem de colunas numéricas (dinheiro)

In [104]:
cols_monetarias = [
    "valorArrecadado",
    "valorAcumuladoConcurso_0_5",
    "valorAcumuladoConcursoEspecial",
    "valorAcumuladoProximoConcurso",
    "valorEstimadoProximoConcurso",
]

for c in cols_monetarias:
    df[c] = pd.to_numeric(df[c], errors="coerce")

### corrigindo booleanos

In [105]:
df["acumulou"] = df["acumulou"].astype("boolean")

### normalizar colunas que são listas

In [106]:
def normalizar_lista(lista):
    if isinstance(lista, list):
        return lista
    if isinstance(lista, str):
        try:
            return ast.literal_eval(lista)
        except Exception as e:
            return None
    return None

In [107]:
df["dezenas"] = df["dezenas"].apply(normalizar_lista)
df["dezenasOrdemSorteio"] = df["dezenasOrdemSorteio"].apply(normalizar_lista)

In [108]:
df["premiacoes"] = df["premiacoes"].apply(normalizar_lista)

In [109]:
df["localGanhadores"] = df["localGanhadores"].apply(normalizar_lista)

### valores ausentes
- concursos sem ganhador (localGanhadores)
- arrecadação sem nada (pode ser nulo)
- premiações vazias (válido)

Esses não precisam ser preenchidos

### dezenas inválidas

In [110]:
select_invalido = df["dezenas"].apply(lambda x: isinstance(x, list) and len(x) != expected_count)

In [111]:
select_invalido.sum()

np.int64(0)

In [112]:
df = df[df["dezenas"].apply(lambda x: isinstance(x, list) and len(x) == expected_count)]

### dezenas fora de 1 a 60

In [113]:
def dezenas_validas(lista, loteria):
    intervalos_dezenas = {
    "megasena": (1, 60),
    "lotofacil": (1, 25),
    "timemania": (1, 80),
    "diadesorte": (1, 31),
    "maismilionaria": (1, 50)
    }
    range_dezenas = intervalos_dezenas[loteria]
    if not isinstance(lista, list):
        return False
    return all(range_dezenas[0] <= int(n) <= range_dezenas[1] for n in lista)

In [114]:
df = df[df.apply(lambda row: dezenas_validas(row["dezenas"], row["loteria"]), axis=1)]

### valor monetário negativo

In [115]:
for coluna in cols_monetarias:
    df = df[(df[coluna].isna()) | (df[coluna] >= 0)]

## Engenharia de features

### separando dia, mês, ano, semana do ano e dia da semana das colunas data e proximoConcurso

In [116]:
df['data_dia'] = df['data'].dt.day
df['data_mes'] = df['data'].dt.month
df['data_ano'] = df['data'].dt.year
df['semana_ano_concurso'] = df['data'].dt.isocalendar().week
df["dia_semana_concurso"] = df["data"].dt.weekday

df['proximoConcurso_dia'] = df['proximoConcurso'].dt.day
df['proximoConcurso_mes'] = df['proximoConcurso'].dt.month
df['proximoConcurso_ano'] = df['proximoConcurso'].dt.year

### Separar cada número das dezenas sorteadas em colunas diferentes

In [117]:
df["qtd_dezenas"] = df["dezenas"].apply(lambda x: len(x) if isinstance(x, list) else 0)
max_dezenas = df["qtd_dezenas"].max()

In [118]:
max_dezenas

np.int64(6)

In [119]:
for i in range(max_dezenas):
    df[f"dezena_{i+1}"] = df["dezenas"].apply(
        lambda x: x[i] if isinstance(x, list) and len(x) > i else None
    )

In [120]:
df.drop(columns=["qtd_dezenas"], inplace=True)

### Separar cidade e estado em colunas diferentes (local)

In [121]:
df[["nome_local", "resto"]] = df["local"].str.split(" em ", n=1, expand=True)

In [122]:
df[["cidade", "estado"]] = df["resto"].str.rsplit(", ", n=1, expand=True)

In [123]:
df.drop(columns=["resto"], inplace=True)

### separar premiações em colunas, criar coluna com quantos premiados

In [124]:
df['premiacoes'][1]

[{'descricao': '6 acertos', 'faixa': 1, 'ganhadores': 0, 'valorPremio': 0},
 {'descricao': '5 acertos',
  'faixa': 2,
  'ganhadores': 74,
  'valorPremio': 18450.77},
 {'descricao': '4 acertos',
  'faixa': 3,
  'ganhadores': 4971,
  'valorPremio': 273.62}]

In [125]:
df["premiacoes"] = df["premiacoes"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str)
    else (x if isinstance(x, list) else None)
)

In [126]:
def expandir_premiacoes(prem_list):
    """
    Recebe a lista de premiações e retorna um dicionário
    com colunas dinâmicas: ganhadores_faixaX e valor_faixaX.
    Funciona para qualquer loteria.
    """
    if not isinstance(prem_list, list):
        return {}

    resultado = {}

    for item in prem_list:
        # identificar faixa
        faixa = item.get("faixa")

        if faixa is None:
            continue

        # criar nomes de coluna dinâmicos
        col_ganhadores = f"ganhadores_faixa_{faixa}"
        col_valor = f"valor_faixa_{faixa}"

        resultado[col_ganhadores] = item.get("ganhadores")
        resultado[col_valor] = item.get("valorPremio")

    return resultado

In [127]:
premios_expandido = df["premiacoes"].apply(expandir_premiacoes)
premios_df = pd.DataFrame(premios_expandido.tolist())
df = pd.concat([df, premios_df], axis=1)

In [128]:
cols_ganhadores = [c for c in df.columns if c.startswith("ganhadores_faixa_")]
df["total_ganhadores"] = df[cols_ganhadores].sum(axis=1)


cols_valores = [c for c in df.columns if c.startswith("valor_faixa_")]
df["total_pago_premios"] = df[cols_valores].sum(axis=1)


df["media_premio_real"] = df["total_pago_premios"] / df["total_ganhadores"]

### localGanhadores separar por coluna

In [129]:
df['localGanhadores'][0]

[{'ganhadores': 1,
  'municipio': 'BOA VISTA',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'RR'}]

In [130]:
def normalizar_local(lista):
    if isinstance(lista, list):
        return lista
    return []

df["localGanhadores"] = df["localGanhadores"].apply(normalizar_local)

In [131]:
df["municipioGanhador"] = df["localGanhadores"].apply(
    lambda x: x[0].get("municipio") if len(x) > 0 else None
)

df["ufGanhador"] = df["localGanhadores"].apply(
    lambda x: x[0].get("uf") if len(x) > 0 else None
)

In [132]:
def is_ticket_online(x):
    if len(x) == 0:
        return False
    registro = x[0]
    municipio = registro.get("municipio", "").strip().upper()
    uf = registro.get("uf", "").strip().upper()

    return municipio == "CANAL ELETRONICO" or uf == "BR"

df["ticketGanhadorOnline"] = df["localGanhadores"].apply(is_ticket_online)

In [133]:
df['ticketGanhadorOnline'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2366 entries, 0 to 2365
Series name: ticketGanhadorOnline
Non-Null Count  Dtype
--------------  -----
2366 non-null   bool 
dtypes: bool(1)
memory usage: 2.4 KB


In [134]:
df['ufGanhador'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2366 entries, 0 to 2365
Series name: ufGanhador
Non-Null Count  Dtype 
--------------  ----- 
496 non-null    object
dtypes: object(1)
memory usage: 18.6+ KB


### verificar diferença entre nulo e 0 em valorArrecadado

In [135]:
df['valorArrecadado']

0       26151116.0
1              0.0
2              0.0
3              0.0
4       66546110.0
           ...    
2361           0.0
2362    60646614.0
2363    94481851.5
2364           0.0
2365           0.0
Name: valorArrecadado, Length: 2366, dtype: float64

In [136]:
df["valorArrecadado"] = df["valorArrecadado"].replace(0, None)

### criar coluna de razão entre valor estimado e valor acumulado

In [137]:
def calcular_razao(row):
    acumulado = row["valorAcumuladoProximoConcurso"]
    estimado  = row["valorEstimadoProximoConcurso"]

    # evita problemas de None, NaN ou divisões por zero
    if acumulado is None or pd.isna(acumulado) or acumulado == 0:
        return None
    if estimado is None or pd.isna(estimado):
        return None

    return estimado / acumulado

df["razaoEstimadoAcumulado"] = df.apply(calcular_razao, axis=1)

### colunas extras

In [138]:
def normalizar_dezenas(lista):
    if not isinstance(lista, list):
        return None
    nova = []
    for item in lista:
        try:
            nova.append(int(item))  # converte "03" → 3
        except:
            return None  # caso algo esteja muito errado
    return nova

df["dezenas"] = df["dezenas"].apply(normalizar_dezenas)

In [139]:
df["qtd_pares"] = df["dezenas"].apply(
    lambda x: sum(1 for n in x if n % 2 == 0) if isinstance(x, list) else None
)

df["qtd_impares"] = df["dezenas"].apply(
    lambda x: sum(1 for n in x if n % 2 != 0) if isinstance(x, list) else None
)


df["range_dezenas"] = df["dezenas"].apply(
    lambda x: max(x) - min(x) if isinstance(x, list) and len(x) == 6 else None
)